In [1]:
# Import necessary libraries
import numpy as np
import pandas as pd
import time
import psutil
import threading
from threading import Thread
import joblib
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
X_train=pd.read_csv('X_train3.csv')
X_test=pd.read_csv('X_test3.csv')
y_train=pd.read_csv('y_train3.csv')
y_test=pd.read_csv('y_test3.csv')
# Create a linear regression model

In [3]:
def get_cpu_and_ram_utilization(pid, duration):
    process = psutil.Process(pid)
    cpu_usages = []
    ram_usages = []
    start_time = time.time()
    while time.time() - start_time < duration:
        cpu_usage = process.cpu_percent(interval=1)
        ram_usage = process.memory_percent()
        cpu_usages.append(cpu_usage)
        ram_usages.append(ram_usage)
        print(f"CPU utilization: {cpu_usage}% | RAM utilization: {ram_usage}%")
    total_cpu_usage = sum(cpu_usages)
    return total_cpu_usage, cpu_usages, ram_usages

In [4]:
pid = os.getpid()
monitor_duration = 3

In [5]:
# Train the model using the training data
results={}


def lin_reg(X_train,y_train):
  start_time = time.time()
  model = LinearRegression()
  model.fit(X_train, y_train)
  end_time = time.time()
  results['training_time']=end_time-start_time
  print(f"Training time: {results['training_time']} seconds")
  results['bias']=model.intercept_
  results['weights']=model.coef_
  results['model']=model


train_thread = Thread(target=lin_reg, args=(X_train, y_train))
train_thread.start()

total_cpu_usage, cpu_usages, ram_usages = get_cpu_and_ram_utilization(pid, monitor_duration)
train_thread.join()

print(f"Total CPU utilization over {monitor_duration} seconds: {total_cpu_usage}%")

# Print the CPU and RAM usage per second
print("CPU usage per second:", cpu_usages)
print("RAM usage per second:", ram_usages)

# Calculate total CPU resource consumption in 'CPU-seconds'
cpu_seconds = sum(cpu_usages) / 100
print(f"Total CPU resource consumption: {cpu_seconds} CPU-seconds")


# Make predictions using the testing data
model=results['model']

test_time_start=time.time()
y_pred = model.predict(X_test)
test_time_end=time.time()

results['outputs']=y_pred

# Evaluate the model
results['testing_time']=test_time_end-test_time_start
mse = mean_squared_error(y_test, results['outputs'])

print(f"Mean Squared Error: {mse}")
print(f"{results['weights']}")
print(f"{results['bias']}")
print(f"{results['training_time']}")
print(f"{results['testing_time']}")

joblib.dump((results['weights'], results['bias'], results['outputs'], cpu_seconds, cpu_usages, ram_usages, results['training_time'], results['testing_time']), 'variables_sklearn.pkl')

Training time: 0.0837709903717041 seconds
CPU utilization: 9.0% | RAM utilization: 1.5751448783563549%
CPU utilization: 1.0% | RAM utilization: 1.5751448783563549%
CPU utilization: 1.0% | RAM utilization: 1.5751448783563549%
Total CPU utilization over 3 seconds: 11.0%
CPU usage per second: [9.0, 1.0, 1.0]
RAM usage per second: [1.5751448783563549, 1.5751448783563549, 1.5751448783563549]
Total CPU resource consumption: 0.11 CPU-seconds
Mean Squared Error: 11.451835701730925
[[-4.18429461e-04 -1.02091686e-01 -1.16759698e+00 -1.52018908e-02
   1.20497553e-01 -1.10131404e-01  1.54643880e-04 -6.16340088e-03
  -2.27576566e-05  4.40880545e-02 -8.90090551e-02  6.58276407e-03
  -2.20623562e-02  1.44304355e-02 -4.47089230e-01  4.60283213e-05
   1.45120816e-04 -7.35272193e-02 -8.08695522e-02  9.69987639e+00
   8.21740949e-01]]
[260.57364161]
0.0837709903717041
0.020354747772216797


['variables_sklearn.pkl']